In [1]:
function pivot!(M::Matrix{T}, row::Int, col::Int) where T
    M[row, :] ./= M[row, col]
    for i in 1:size(M, 1)
        if i != row
            M[i, :] .-= M[i, col] .* M[row, :]
        end
    end
end

function findEnteringVariable(M::Matrix{T}) where T
    cost = M[end, 1:end-1]
    indices = findall(x -> x < 0, cost)
    return isempty(indices) ? -1 : indices[1]
end

function findLeavingVariable(M::Matrix{T}, entering_var::Int) where T
    min_ratio = Inf
    leaving_var = 0
    for i in 1:size(M, 1) - 1
        if M[i, entering_var] < 0
            ratio = -M[i, end] / M[i, entering_var]
            if ratio < min_ratio
                min_ratio = ratio
                leaving_var = i
            end
        end
    end
    return (min_ratio == Inf) ? -1 : leaving_var
end

function dualSimplexSolver(A::Matrix{T}, b::Vector{T}, c::Vector{T}; verbose::Bool=false) where T
    M = [A b; c' T(0)]
    
    while true
            
        if verbose
            display(M)
        end
        
        # Trouver la variable entrante 
        entering = findEnteringVariable(M)
        if entering == -1
            break  # Solution optimale trouvée
        end
        
        # Trouver la variable sortante
        leaving = findLeavingVariable(M, entering)
        if leaving == -1
            error("Le problème est non borné.")
        end

        # Executer le pivot
        if verbose
            println("Entrée x$entering remplace sortie x$leaving")
        end
        pivot!(M, leaving, entering)
    end
    
    return M[1:end-1, end], M[end, end]  # Renvoie la solution et la valeur de l'objectif
end


c = [-3, -4, -5, 0, 0, 0]


A = [[1  0  1  0  0  -1] 
     [0  1  2  1  0  -1]  
     [0  0  0  1  0  -1]] 

b = [3, 4, 2]

solution, objective_value = dualSimplexSolver(A, b, c, verbose=true)

println("Solution: ", solution)
println("Valeur de l’objectif: ", objective_value)

4×7 Matrix{Int64}:
  1   0   1  0  0  -1  3
  0   1   2  1  0  -1  4
  0   0   0  1  0  -1  2
 -3  -4  -5  0  0   0  0

LoadError: Le problème est non borné.